In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import warnings
import lightgbm as lgb
import gc
import plotly.express as px
from sklearn.metrics import mean_absolute_error

warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
Df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")

In [3]:
Df = Df[~((Df["stock_id"] == 131) & (Df["date_id"] == 35) | (Df["stock_id"] == 101) & (Df["date_id"] == 328) | (Df["stock_id"] == 158) & (Df["date_id"] == 388) | (Df["stock_id"] == 19) & (Df["date_id"] == 438))]

In [4]:
Df.reset_index(drop=True, inplace=True)

In [5]:
df = Df.copy()

In [6]:
sizes = ["imbalance_size", "matched_size", "bid_size", "ask_size"]
for feat in sizes:
    df[feat] = np.log(df[feat])

In [7]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]

weights = {int(k):v for k,v in enumerate(weights)}

In [8]:
df["pre_target"] = df.groupby("stock_id")["target"].shift(55)
df["pre_target_std"] = df.groupby(["stock_id", "date_id"])["pre_target"].transform("std")
df = df.drop("pre_target", axis=1)
df["pre_ask"] = df.groupby("stock_id")["ask_price"].shift(55)
df["pre_ask_std"] = df.groupby(["stock_id", "date_id"])["pre_ask"].transform("std")
df = df.drop("pre_ask", axis=1)
df["pre_bid"] = df.groupby("stock_id")["bid_price"].shift(55)
df["pre_bid_std"] = df.groupby(["stock_id", "date_id"])["pre_bid"].transform("std")
df = df.drop("pre_bid", axis=1)
df["pre_wap"] = df.groupby("stock_id")["wap"].shift(55)
df["pre_wap_std"] = df.groupby(["stock_id", "date_id"])["pre_wap"].transform("std")
df["pre_wap"] = df.groupby(["stock_id", "date_id"])["pre_wap"].transform("mean")

In [9]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [10]:
global_stock_id_feats = {
        "median_size": df.groupby("stock_id")["bid_size"].median() + df.groupby("stock_id")["ask_size"].median(),
        "std_size": df.groupby("stock_id")["bid_size"].std() + df.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df.groupby("stock_id")["bid_size"].max() - df.groupby("stock_id")["bid_size"].min(),
        "median_price": df.groupby("stock_id")["bid_price"].median() + df.groupby("stock_id")["ask_price"].median(),
        "std_price": df.groupby("stock_id")["bid_price"].std() + df.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df.groupby("stock_id")["ask_price"].min() - df.groupby("stock_id")["bid_price"].max(),
    }

In [11]:
def imbalance_features(df):
    prices = ["reference_price", "far_price", "near_price", "bid_price", "ask_price", "wap"]
    sizes = ["imbalance_size", "matched_size", "bid_size", "ask_size"]

    df["volume"] = df.eval("bid_size + ask_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size - ask_size) / (bid_size + ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size - matched_size)/(matched_size + imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    df["price_spread"] = df.eval("ask_price - bid_price")
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df.eval("stock_weights * wap")
    
    grouped = df.groupby(["stock_id", "date_id"])
    #cumulative features
    df["cum_cnt"] = grouped.cumcount() + 1
    df["bid_cum_sum"] = grouped["bid_price"].cumsum()
    df["bid_price_sq"] = df.eval("bid_price ** 2")
    df["bid_sq_cum_sum"] = df.groupby(['stock_id', "date_id"])["bid_price_sq"].cumsum()
    df["bid_price_std"] = np.sqrt((df["bid_sq_cum_sum"] - df["bid_cum_sum"] ** 2 / df["cum_cnt"]) / df["cum_cnt"])
    df = df.drop(["bid_cum_sum", "bid_price_sq", "bid_sq_cum_sum"], axis=1)
    df["ask_cum_sum"] = grouped["ask_price"].cumsum()
    df["ask_price_sq"] = df.eval("ask_price ** 2")
    df["ask_sq_cum_sum"] = df.groupby(['stock_id', "date_id"])["ask_price_sq"].cumsum()
    df["ask_price_std"] = np.sqrt((df["ask_sq_cum_sum"] - df["ask_cum_sum"] ** 2 / df["cum_cnt"]) / df["cum_cnt"])
    df = df.drop(["ask_cum_sum", "ask_price_sq", "ask_sq_cum_sum"], axis=1)
    df["wap_cum_sum"] = grouped["wap"].cumsum()
    df["wap_sq"] = df.eval("wap ** 2")
    df["wap_sq_cum_sum"] = df.groupby(['stock_id', "date_id"])["wap_sq"].cumsum()
    df["wap_std"] = np.sqrt((df["wap_sq_cum_sum"] - df["wap_cum_sum"] ** 2 / df["cum_cnt"]) / df["cum_cnt"])
    df["volume_cum"] = grouped["volume"].cumsum()
    df["volume_sq"] = df.eval("volume_cum ** 2")
    df["volume_sq_cum_sum"] = df.groupby(['stock_id', "date_id"])["volume_sq"].cumsum()
    df["volume_mean"] = df.eval("volume_cum / cum_cnt")
    df["volume_std"] = np.sqrt((df["volume_sq_cum_sum"] - df["volume_cum"] ** 2 / df["cum_cnt"]) / df["cum_cnt"])
    df["new_liquidity_imbalance"] = df.eval("(bid_size - ask_size) / (volume_mean)")
    df["price_spread_cum"] = grouped["price_spread"].cumsum()
    df["price_spread_mean"] =  df.eval("price_spread_cum / cum_cnt")
    df["price_spread_ratio"] = df.eval("price_spread / price_spread_mean")
    df["new_market_urgency"] = df.eval("price_spread_ratio * new_liquidity_imbalance")
    df = df.drop(["volume_cum", "volume_sq", "volume_sq_cum_sum", "price_spread_cum"], axis=1)
    
    
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
    
    
    # time features
    df["imbalance_momentum"] = df.groupby("stock_id")["imbalance_size"].diff(periods=1) / df["matched_size"]
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby("stock_id")["price_spread"].diff()
    df["mid_price_movement"] = df.groupby("stock_id")["mid_price"].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

    df["price_pressure"] = df["imbalance_size"] * (df["ask_price"] - df["bid_price"])
    df["market_urgency"] = df["price_spread"] * df["liquidity_imbalance"]
    df["depth_pressure"] = (df["ask_size"] - df["bid_size"]) * (df["far_price"] - df["near_price"])
    df["spread_depth_ratio"] = (df["ask_price"] - df["bid_price"]) / (df["bid_size"] + df["ask_size"])
    df["relative_spread'"] = (df["ask_price"] - df["bid_price"]) / df["wap"]
        
    grouped = df.groupby(["stock_id", "date_id"])
    # time lag
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_shift_{window}"] = grouped[col].shift(window)
            df[f"{col}_ret_{window}"] = grouped[col].pct_change(window)
            df[f"{col}_liq_{window}"] = df[f"{col}_ret_{window}"] * df["liquidity_imbalance"]
    
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'wap', 'near_price', 'far_price']:
        for window in [1, 2, 3, 5, 10]:
            df[f"{col}_diff_{window}"] = grouped[col].diff(window)
    
    df["height_imb"] = df.eval("(bid_price_diff_1 - ask_price_diff_1) / (bid_price_diff_1 + ask_price_diff_1)")
    
    return df.replace([np.inf, -np.inf], 0)

In [12]:
def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

In [13]:
def other_features(df):
    df["seconds"] = df["seconds_in_bucket"] % 60
    df["minute"] = df["seconds_in_bucket"] // 60
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

In [14]:
def generate_all_features(df):
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target", "currently_scored"]]
    df = df[cols]
    df = imbalance_features(df)
    df = numba_imb_features(df)
    df = other_features(df)
    gc.collect()
    
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id", "currently_scored"]]
    
    return df[feature_name]

In [15]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """

    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    return df

In [16]:
split_day = 435
df_train = df[df["date_id"] <= split_day]
df_valid = df[df["date_id"] > split_day]

In [17]:
df_train_feats = generate_all_features(df_train)
df_train_feats = reduce_mem_usage(df_train_feats, 1)
df_valid_feats = generate_all_features(df_valid)
df_valid_feats = reduce_mem_usage(df_valid_feats)

Memory usage of dataframe is 6621.82 MB
Memory usage after optimization is: 3243.06 MB
Decreased by 51.02%


In [18]:
feature_name = list(df_train_feats.columns)

In [19]:
# lgb_params = {
#         "objective" : "mae",
#         "n_estimators" : 5000,
#         "num_leaves" : 128,
#         "subsample" : 0.6,
#         "colsample_bytree" : 0.6,
#         "learning_rate" : 0.02,
#         "n_jobs" : 4,
#         "device" : "gpu",
#         "verbosity": -1,
#         "importance_type" : "gain",
#     }

In [20]:
lgb_params = {
        "objective": "mae",
        "n_estimators": 6000,
        "num_leaves": 256,
        "subsample": 0.6,
        "colsample_bytree": 0.8,
        "learning_rate": 0.0086,
        'max_depth': 11,
        "n_jobs": -1,
        "device": "gpu",
        "verbosity": -1,
        "importance_type": "gain",
        "reg_alpha": 0.2,
        "reg_lambda": 3.25
    }

In [21]:
num_folds = 5
fold_size = 480 // num_folds
gap = 5

models = []
models_cbt = []
scores = []
improvements = []
date_ids = df_train['date_id'].values

for i in range(num_folds):
    start = i * fold_size
    end = start + fold_size
    
    # Define the purged set ranges
    purged_before_start = start - 2
    purged_before_end = start + 2
    purged_after_start = end - 2
    purged_after_end = end + 2
    
    # Exclude the purged ranges from the test set
    purged_set = ((date_ids >= purged_before_start) & (date_ids <= purged_before_end)) | ((date_ids >= purged_after_start) & (date_ids <= purged_after_end))
    
    # Define test_indices excluding the purged set
    test_indices = (date_ids >= start) & (date_ids < end) & ~purged_set
    train_indices = ~test_indices & ~purged_set
    
    df_fold_train = df_train_feats[train_indices]
    df_fold_train_target = df_train['target'][train_indices]
    df_fold_valid = df_train_feats[test_indices]
    df_fold_valid_target = df_train['target'][test_indices]

    print(f"Fold {i+1} Model Training")
    
    # Train a LightGBM model for the current fold
    lgb_model = lgb.LGBMRegressor(**lgb_params)
    lgb_model.fit(
        df_fold_train[feature_name],
        df_fold_train_target,
        eval_set=[(df_fold_valid[feature_name], df_fold_valid_target)],
        callbacks=[
            lgb.callback.early_stopping(stopping_rounds=100),
            lgb.callback.log_evaluation(period=100),
        ],
    )

    # Append the model to the list
    models.append(lgb_model)
#     # Save the model to a file
#     model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
#     lgb_model.booster_.save_model(model_filename)
#     print(f"Model for fold {i+1} saved to {model_filename}")

    # Evaluate model performance on the validation set
    fold_predictions = lgb_model.predict(df_fold_valid[feature_name])
    fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
    scores.append(fold_score)
    improve = 1 - fold_score / np.mean(abs(df_fold_valid_target))
    improvements.append(improve)
    print(f"Fold {i+1} MAE: {fold_score}")
    print(f"Fold {i+1} improvement: {improve}")

    # Free up memory by deleting fold specific variables
    del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
    gc.collect()

# Calculate the average best iteration from all regular folds
average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

# Update the lgb_params with the average best iteration
final_model_params = lgb_params.copy()
final_model_params["n_estimators"] = average_best_iteration

print(f"Training final model with average best iteration: {average_best_iteration}")

# Train the final model on the entire dataset
final_model = lgb.LGBMRegressor(**final_model_params)
final_model.fit(
    df_train_feats[feature_name],
    df_train["target"],
    eval_set=[(df_valid_feats[feature_name], df_valid["target"])],
    callbacks=[
        lgb.callback.log_evaluation(period=100),
    ],
)

# Append the final model to the list of models
models.append(final_model)

fold_predictions = lgb_model.predict(df_valid_feats[feature_name])
fold_score = mean_absolute_error(fold_predictions, df_valid["target"])
scores.append(fold_score)
improve = 1 - fold_score / np.mean(abs(df_valid["target"]))
improvements.append(improve)
# Save the final model to a file
# final_model_filename = os.path.join(model_save_path, 'doblez-conjunto.txt')
# final_model.booster_.save_model(final_model_filename)
# print(f"Final model saved to {final_model_filename}")

# Now 'models' holds the trained models for each fold and 'scores' holds the validation scores
print(f"Average MAE across all folds: {np.mean(scores)}")

Fold 1 Model Training
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 5.60906
[200]	valid_0's l1: 5.57901
[300]	valid_0's l1: 5.5667
[400]	valid_0's l1: 5.55877
[500]	valid_0's l1: 5.55356
[600]	valid_0's l1: 5.55033
[700]	valid_0's l1: 5.54773
[800]	valid_0's l1: 5.54574
[900]	valid_0's l1: 5.54417
[1000]	valid_0's l1: 5.54309
[1100]	valid_0's l1: 5.54227
[1200]	valid_0's l1: 5.5417
[1300]	valid_0's l1: 5.54097
[1400]	valid_0's l1: 5.5406
[1500]	valid_0's l1: 5.54039
[1600]	valid_0's l1: 5.54013
[1700]	valid_0's l1: 5.5399
[1800]	valid_0's l1: 5.53973
[1900]	valid_0's l1: 5.53954
[2000]	valid_0's l1: 5.53937
[2100]	valid_0's l1: 5.53918
[2200]	valid_0's l1: 5.53903
[2300]	valid_0's l1: 5.5389
[2400]	valid_0's l1: 5.53877
[2500]	valid_0's l1: 5.53871
[2600]	valid_0's l1: 5.53865
[2700]	valid_0's l1: 5.53854
[2800]	valid_0's l1: 5.53837
[2900]	valid_0's l1: 5.53822
[3000]	valid_0's l1: 5.53818
[3100]	valid_0's l1: 5.53814
[3200]	valid_0's l1: 5.53805
[3

In [22]:
feat_imp = pd.Series(final_model.feature_importances_, index=feature_name).sort_values()
import_feat = feat_imp[feat_imp>10000].index
fig = px.bar(x=feat_imp[import_feat], y=import_feat, orientation='h')
fig.show()

In [23]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [24]:
y_min, y_max = -64, 64
cache = pd.DataFrame()
pre_information = pd.DataFrame({"stock_id": range(200), "pre_target_std": np.nan, "pre_ask_std": np.nan,
                                "pre_bid_std": np.nan, "pre_wap": np.nan, "pre_wap_std": np.nan})
model_weights = improvements / sum(improvements)
print(f"Weights of models are:")
print(model_weights)

for (test, revealed_targets, sample_prediction) in iter_test:
    for term in sizes:
        test[term] = np.log(test[term])
    if test["seconds_in_bucket"].unique()[0] == 0:
        pre_information["pre_target_std"] = revealed_targets.groupby("stock_id")["revealed_target"].std()
        if not cache.empty:
            pre_information["pre_ask_std"] = cache.groupby("stock_id")["ask_price"].std()
            pre_information["pre_bid_std"] = cache.groupby("stock_id")["bid_price"].std()
            pre_information["pre_wap"] = cache.groupby("stock_id")["wap"].mean()
            pre_information["pre_wap_std"] = cache.groupby("stock_id")["wap"].std()
        
        cache = pd.DataFrame()
    cache = pd.concat([cache, test], ignore_index=True, axis=0)
    temp = pd.merge(cache, pre_information, on='stock_id', how='left')
    feat = generate_all_features(temp)[-len(test):]
    feat = reduce_mem_usage(feat)
    
    lgb_prediction = np.zeros(len(test))
    for model, weight in zip(models, model_weights):
        lgb_prediction += weight * model.predict(feat)
        
    lgb_prediction = lgb_prediction - np.mean(lgb_prediction)
    clipped_predictions = np.clip(lgb_prediction, y_min, y_max)
    sample_prediction['target'] = clipped_predictions
    env.predict(sample_prediction)

Weights of models are:
[0.20762835 0.20518899 0.1513614  0.14499882 0.15307313 0.1377493 ]
This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
